In [1]:
#Library imports
import os
import pandas as pd
import numpy as np

import torch
import torch.nn as nn  
import torch.optim as optim  
import torchvision.transforms as transforms  
import torchvision
from torch.utils.data import (Dataset,DataLoader) 

from PIL import Image
from skimage import io

In [2]:
#Get working directory
directory = os.getcwd() # Path of the current working directory.
print(directory)
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f" {device} being used")
#if cpu is being used with PyTorch training will be slower

C:\Users\Andreas\Desktop\ML\Final
 cpu being used


In [3]:
#Hyperparameters
num_epochs = 5 #complete passess over the data set
num_classes = 2 #male/female or smile/no-smile
batch_size = 100 #data must be loaded in batches for more efficient training (high batch size can lead to memory overload)
learning_rate = 0.01

In [4]:
#transformations on the images
#transformer=transforms.Compose([transforms.Resize((224,224)), #make sure all images are the same size
#                                transforms.ToPILImage(),
#                                transforms.RandomHorizontalFlip(), #0.5 chance for image to be flipped to increase uniqueness x2
#                                transforms.ToTensor(),#pixel rate from 0-255 to 0-1 as PyTorch only reads tensors
#                                transforms.Normalize([0.5,0.5,0.5], #change range from 0-1 to (-1)-1 for RGB channel
#                                                     [0.5,0.5,0.5]) #formula:(x-mean)/std
#]) 

In [5]:
# class to load the custom dataset

class celebaDataset(Dataset):
    def __init__(self,csv_file,root_dir,transform=None):
        self.annotations=pd.read_csv(csv_file)
        self.root_dir=root_dir
        self.transform=transform
        pass
  
    def __len__(self):
        
        return len(self.annotations)
        pass
    def __getitem__(self, index): #PyTorch chooses the index
        img_path=os.path.join(self.root_dir, self.annotations.iloc[index,0]) #row 'index' and column 0
        image=io.imread(img_path)
        y_label=torch.tensor(int(self.annotations.iloc[index,1]))
        y=(y_label+1)/2
        yt=y.type(torch.LongTensor)
        if self.transform:
            image=self.transform(image)
        return (image,yt)
        
        pass

In [7]:
train_dataset=celebaDataset(csv_file='dataset_AMLS_22-23/celeba/gender_labels.csv',
                            root_dir='dataset_AMLS_22-23/celeba/img',transform=transforms.ToTensor())
train_loader=DataLoader(dataset=train_dataset,batch_size=100,shuffle=True)
#shuffle=True to ensure our model is not biased for some categories 
test_dataset=celebaDataset(csv_file='dataset_AMLS_22-23_test/celeba_test/gender_labels_test.csv',
                            root_dir='dataset_AMLS_22-23_test/celeba_test/img_test',transform=transforms.ToTensor())
test_loader=DataLoader(dataset=test_dataset,batch_size=100,shuffle=True)
print('Train dataset size:', len(train_dataset)) #must give 5000
print('Test dataset size:', len(test_dataset))

Train dataset size: 5000
Test dataset size: 1000


In [8]:
#CNN model
class convNet(nn.Module):
    def __init__(self,num_classes=2):
        super(convNet,self).__init__()
        
        #Input shape =(batch_size,RGB_channel,Image dimensions)=(100,3,178,218)
        #The first convolutional layer
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #New shape from formula (width-kernel+2P)/s +1 ->Shape=(100,12,178,218)
        self.bn1=nn.BatchNorm2d(num_features=12) #normalisation
        #Shape=(100,12,178,218)
        self.relu1=nn.ReLU()
        #Shape=(100,12,178,218)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (100,12,89,109)
        
        #second convolutional layer
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (100,20,89,109)
        self.relu2=nn.ReLU()
        #Shape= (100,20,89,109)
        
        
        #Third convolutional layer
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (100,32,89,109)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (100,32,89,109)
        self.relu3=nn.ReLU()
        #Shape= (100,32,89,109)
        
        #fully connected layer
        self.fc=nn.Linear(in_features=89 * 109 * 32,out_features=num_classes)
        
        
        #feed forward function
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
        #Above output will be in matrix form, with shape (100,32,112,112)
            
        output=output.view(-1,32*89*109)
            
            
        output=self.fc(output)
            
        return output
            
        

In [9]:
model=convNet(num_classes=2).to(device) #send it to cuda/cpu

In [10]:
#Loss and optimizer functions
optimizer=torch.optim.Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
criterion=nn.CrossEntropyLoss()

In [12]:
# Train Network

for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")

# Check accuracy on training to see how good our model is
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()


print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)
    
print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)    
    

Cost at epoch 0 is 0.026011550644616365
Cost at epoch 1 is 0.025158186147455127
Cost at epoch 2 is 0.005120548393852005
Cost at epoch 3 is 0.0016445697624112654
Cost at epoch 4 is 0.0018914735615112477
Checking accuracy on Training Set
Got 5000 / 5000 with accuracy 100.00
Checking accuracy on Test Set
Got 920 / 1000 with accuracy 92.00
